[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kamerlinlab/KIF/blob/main/tutorials/Tutorial_KE07_Regression_ML_Stats.ipynb)

### Tutorial: Regression ML and Statistical Analysis on the R1 and R4 KE07s

In this jupyter notebook we will use the model_building.py and stat_modelling.py modules to perform machine learning (ML) and statisical analysis (both regression) on a kemp eliminase enzyme (KE07). Our target variable is a continous value (hence regression) and is the value of the KE07's W50 Chi2 angle. 

This notebook will also cover all the pre- and post-processing steps requireds to prepare, analyse and visualise the results.

The dataset used here is for the R1 and R4 KE07s and is the same data as what was used in the manuscript. 

<center><img src="https://raw.githubusercontent.com/kamerlinlab/KIF/main/tutorials/miscellaneous/ke07_banner.png" style="width: 70%" /></center>


### Setup

Install and load the required modules and then download the dataset we'll be working on from google drive

In [21]:
%pip install KIF

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd

from key_interactions_finder import data_preperation
from key_interactions_finder import stat_modelling
from key_interactions_finder import model_building
from key_interactions_finder import post_proccessing
from key_interactions_finder import pymol_projections
from key_interactions_finder import chimerax_projections

We will first need to donwload the dataset from google drive. 
The tutorial data will be saved in the relative path defined by "save_dir" in the cell block below.

You can change this as you see fit. If you want to use the current directory you can do:

save_dir=""

In [23]:
from key_interactions_finder.utils import download_prep_tutorial_dataset

drive_url = r"https://drive.google.com/file/d/1pqFUMMjt9gDYOxtkVpDmyXwKXiHp0wFi/view?usp=share_link"
save_dir = "tutorial_datasets/"

download_prep_tutorial_dataset(drive_url=drive_url, save_dir=save_dir)

Downloading...
From: https://drive.google.com/uc?id=1pqFUMMjt9gDYOxtkVpDmyXwKXiHp0wFi
To: /home/roryc/Desktop/git_projects/KIF/tutorials/tutorial_datasets/tutorial_dataset.zip
100%|██████████| 22.4M/22.4M [00:02<00:00, 8.48MB/s]


Tutorial files were successfully downloaded and unzipped.


In [24]:
# Where all input data is stored. 
in_dir = save_dir + r"KE07_Tutorial/Input_data/"

# The target variable's per frame values are stored here. 
target_file = in_dir + r"R1_5d2w_1in10_Trp50_Chi2.dat"

# Path to the variable we will use to filter frames with (optional addition for this system). 
w50_chi1_file = in_dir + r"R1_5d2w_1in10_Trp50_Chi1.dat"

# The pdb file will later be used to help make the ChimeraX visualisations of the results.  
pdb_file = in_dir + r"R1_5d2w.pdb"

# output folders
stats_out_dir = save_dir + r"KE07_Tutorial/KE07_stat_analysis" 
ml_out_dir = save_dir + r"KE07_Tutorial/KE07_ml_analysis"

### Preperation Step 1: Load the non-covalent interaction datasets

The contact identification calculation was split into 4 blocks of different residues ranges. We will first need to load these blocks in and merge them. Luckly this is very easy with pandas. 

Note this data was generated using the script: "identify_contacts.py" which is provided with KIF.

In [25]:
input_files = ["KE07_block1.csv", "KE07_block2.csv", "KE07_block3.csv", "KE07_block4.csv"]
dfs = []
for file_name in input_files:
    file_path = in_dir + file_name
    df = pd.read_csv(file_path)
    dfs.append(df)

all_contacts_df = pd.concat(dfs, join='outer', axis=1)
all_contacts_df.head(3)

1Ala 4Lys Hbond  1Ala 5Arg Other  1Ala 219Asp Other  1Ala 247Asn Hbond  \
0           0.0009              0.0                0.0             0.5091   
1           0.0000              0.0                0.0             0.3099   
2           0.0000              0.0                0.0             1.1025   

   2Leu 247Asn Hbond  3Ala 219Asp Other  3Ala 247Asn Other  \
0             1.4173             0.1942             8.4905   
1             0.6169             0.1086             8.7308   
2             4.1349             0.1174             4.0820   

   3Ala 248Val Hydrophobic  3Ala 249Arg Hbond  4Lys 45Asp Saltbr  ...  \
0                   0.2583             0.8390             0.9887  ...   
1                   0.0250             0.1596             0.4652  ...   
2                   0.0004             0.0207             0.6851  ...   

   240Tyr 243Lys Other  240Tyr 244His Hbond  241Leu 244His Other  \
0               4.7718              13.4418               4.6074   
1               5.9753              17.8924               5.3730   
2               4.2700              11.3290               4.1186   

   241Leu 245Gly Other  241Leu 246Val Hydrophobic  241Leu 248Val Hydrophobic  \
0               1.8850                     6.7067                     0.6970   
1               1.3674                     7.9847                     1.3530   
2               2.4458                     5.9295                     2.4387   

   242Lys 245Gly Other  242Lys 246Val Other  242Lys 248Val Hbond  \
0               2.3419               1.1636               1.4529   
1               2.4137               1.2101               0.8352   
2               2.8741               0.1052               0.4285   

   242Lys 250Leu Hbond  
0               5.2773  
1               3.9203  
2               3.5174  

[3 rows x 839 columns]

We can see we now have a dataframe with all the contacts found (839) identified and of length 10001, with matches with the number of frames in the trajectory. 

In [26]:
all_contacts_df.shape

(10001, 839)

Now we will add one more row to our dataframe which corresponds to the W50 Chi1 angle. This will allow us to filter the dataframe to remove those frames that belong to  "Conformation B". The reasons for this is  described in full in the paper, but this is essentially to help focus the analysis on the differences between the "A" and "C" state. 
- Note the target variable is the W50 Chi**2** angle, and we will filter data on the Chi**1** angle.

In [27]:
just_chi1_df = pd.read_csv(w50_chi1_file)
just_chi1_df = just_chi1_df.set_axis(["W50Chi1"], axis=1)

chi1_df = pd.concat([just_chi1_df, all_contacts_df], axis=1)
chi1_df.head(3)

W50Chi1  1Ala 4Lys Hbond  1Ala 5Arg Other  1Ala 219Asp Other  \
0  193.934           0.0009              0.0                0.0   
1  188.050           0.0000              0.0                0.0   
2  184.593           0.0000              0.0                0.0   

   1Ala 247Asn Hbond  2Leu 247Asn Hbond  3Ala 219Asp Other  3Ala 247Asn Other  \
0             0.5091             1.4173             0.1942             8.4905   
1             0.3099             0.6169             0.1086             8.7308   
2             1.1025             4.1349             0.1174             4.0820   

   3Ala 248Val Hydrophobic  3Ala 249Arg Hbond  ...  240Tyr 243Lys Other  \
0                   0.2583             0.8390  ...               4.7718   
1                   0.0250             0.1596  ...               5.9753   
2                   0.0004             0.0207  ...               4.2700   

   240Tyr 244His Hbond  241Leu 244His Other  241Leu 245Gly Other  \
0              13.4418               4.6074               1.8850   
1              17.8924               5.3730               1.3674   
2              11.3290               4.1186               2.4458   

   241Leu 246Val Hydrophobic  241Leu 248Val Hydrophobic  242Lys 245Gly Other  \
0                     6.7067                     0.6970               2.3419   
1                     7.9847                     1.3530               2.4137   
2                     5.9295                     2.4387               2.8741   

   242Lys 246Val Other  242Lys 248Val Hbond  242Lys 250Leu Hbond  
0               1.1636               1.4529               5.2773  
1               1.2101               0.8352               3.9203  
2               0.1052               0.4285               3.5174  

[3 rows x 840 columns]

### Preperation Step 2. Prepare the Dataset for calculations with the data_preperation.py module. 

In this step, we take our dataframe and merge our per frame target file to it.

We can also optionally perform several forms of filtering on the non-covalent interactions identified to select what types of interactions we would like to study.  

In [28]:
# Generate supervised dataset instance as we have a target variable.  
supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=chi1_df,
    target_file=target_file,
    is_classification=False,
    header_present=True 
)

supervised_dataset.df_processed.head(3)

Your PyContact features and target variable have been succesufully merged.
You can access this dataset through the class attribute: '.df_processed'.


Target  W50Chi1  1Ala 4Lys Hbond  1Ala 5Arg Other  1Ala 219Asp Other  \
0   85.8483  193.934           0.0009              0.0                0.0   
1   93.7505  188.050           0.0000              0.0                0.0   
2  102.0940  184.593           0.0000              0.0                0.0   

   1Ala 247Asn Hbond  2Leu 247Asn Hbond  3Ala 219Asp Other  3Ala 247Asn Other  \
0             0.5091             1.4173             0.1942             8.4905   
1             0.3099             0.6169             0.1086             8.7308   
2             1.1025             4.1349             0.1174             4.0820   

   3Ala 248Val Hydrophobic  ...  240Tyr 243Lys Other  240Tyr 244His Hbond  \
0                   0.2583  ...               4.7718              13.4418   
1                   0.0250  ...               5.9753              17.8924   
2                   0.0004  ...               4.2700              11.3290   

   241Leu 244His Other  241Leu 245Gly Other  241Leu 246Val Hydrophobic  \
0               4.6074               1.8850                     6.7067   
1               5.3730               1.3674                     7.9847   
2               4.1186               2.4458                     5.9295   

   241Leu 248Val Hydrophobic  242Lys 245Gly Other  242Lys 246Val Other  \
0                     0.6970               2.3419               1.1636   
1                     1.3530               2.4137               1.2101   
2                     2.4387               2.8741               0.1052   

   242Lys 248Val Hbond  242Lys 250Leu Hbond  
0               1.4529               5.2773  
1               0.8352               3.9203  
2               0.4285               3.5174  

[3 rows x 841 columns]

##### Optional Feature Filtering

In the above dataframe we have 1887 columns (so 989 features + 1 target). We can take all of these forward for the stastical analysis or we can perform some filtering in advance (the choice is yours). 
There are five built in filtering methods available to you to perform filtering:

1. **filter_by_occupancy(min_occupancy)** - Remove features that have an %occupancy less than the provided cut-off. %Occupancy is the % of frames with a non 0 value, i.e. the interaction is present in that frame.

2. **filter_by_interaction_type(interaction_types_included)** - Inteactions are defined as one of four possible types: ("Hbond", "Saltbr", "Hydrophobic", "Other"). You select the interactions you want to include.

3. **filter_by_avg_strength(average_strength_cut_off)** - Filter by the per frame contact score/strength for each interaction. You can filter features by the average score. Values below the cut-off are removed. 

4. **filter_by_occupancy_by_class(min_occupancy)** - Special alternative to the the standard filter features by occupancy method. %occupancy is determined for each class (as opposed to whole dataset), meaning only observations from 1 class have to meet the cut-off to keep the feature. Only avaible to datasets with a categorical target variable (classification). 


Finally if at any point in time you want to reset any filtering you've already performed, you can use the following method: 

5. **reset_filtering()** 


In [29]:
supervised_dataset.reset_filtering() 
print(f"Number of features before any filtering: {len(supervised_dataset.df_processed.columns) - 1}")

# Features with a %occupancy of less than 25% are removed. 
supervised_dataset.filter_by_occupancy(min_occupancy=25)
print(f"Number of features after filtering by occupancy: {len(supervised_dataset.df_filtered.columns) - 1}")

# Features with an average interaction strength less than 0.5 will be removed. 
supervised_dataset.filter_by_avg_strength(
    average_strength_cut_off=0.5,  
)
print(f"Number of features after filtering by average interaction scores: {len(supervised_dataset.df_filtered.columns) - 1}")

Number of features before any filtering: 840
Number of features after filtering by occupancy: 830
Number of features after filtering by average interaction scores: 684


Now if we look at the class attributes of our SupervisedFeatureData() instance (we called it: supervised_dataset) using the special "\_\_dict__" method we can see two dataframes we could use in the stastical analysis to follow. 

In [30]:
supervised_dataset.__dict__.keys()

dict_keys(['input_df', 'is_classification', 'target_file', 'header_present', 'df_processed', 'df_filtered'])

They are: 
- 'df_processed' - The unfiltered dataframe, 1700 features
- 'df_filtered' - The filtered dataframe. Less than 1700 features. 

In the following sections we will use the filtered dataframe but either dataframe could be justified based on your goals. 

As described above, we will filter frames to remove those that belong to conformation B, using the W50 chi1 angle to define this.

In [31]:
# Remove all frames with W50 Chi1: < 160 and > 240. 
chi1_chi2_df = supervised_dataset.df_filtered
filtered_df = chi1_chi2_df[( (chi1_chi2_df.W50Chi1 > 160) & (chi1_chi2_df.W50Chi1 < 240) )]

# Now remove "W50Chi1" as its jobs is done. 
df_ready = (filtered_df.drop("W50Chi1", axis=1)).reset_index(drop=True) 

print(f"Rows before filtering by W50 Chi1: {len(supervised_dataset.df_filtered)}")
print(f"Rows after filtering by W50 Chi1: {len(df_ready)}")
df_ready.head(3)

Rows before filtering by W50 Chi1: 10001
Rows after filtering by W50 Chi1: 9281


Target  1Ala 247Asn Hbond  2Leu 247Asn Hbond  3Ala 247Asn Other  \
0   85.8483             0.5091             1.4173             8.4905   
1   93.7505             0.3099             0.6169             8.7308   
2  102.0940             1.1025             4.1349             4.0820   

   3Ala 248Val Hydrophobic  3Ala 249Arg Hbond  4Lys 45Asp Saltbr  \
0                   0.2583             0.8390             0.9887   
1                   0.0250             0.1596             0.4652   
2                   0.0004             0.0207             0.6851   

   4Lys 214Phe Hbond  4Lys 218Ala Hbond  4Lys 219Asp Saltbr  ...  \
0             6.2291             3.6381              5.2280  ...   
1             6.7166             3.5377              5.8784  ...   
2             7.3353             3.8191              6.3520  ...   

   240Tyr 243Lys Other  240Tyr 244His Hbond  241Leu 244His Other  \
0               4.7718              13.4418               4.6074   
1               5.9753              17.8924               5.3730   
2               4.2700              11.3290               4.1186   

   241Leu 245Gly Other  241Leu 246Val Hydrophobic  241Leu 248Val Hydrophobic  \
0               1.8850                     6.7067                     0.6970   
1               1.3674                     7.9847                     1.3530   
2               2.4458                     5.9295                     2.4387   

   242Lys 245Gly Other  242Lys 246Val Other  242Lys 248Val Hbond  \
0               2.3419               1.1636               1.4529   
1               2.4137               1.2101               0.8352   
2               2.8741               0.1052               0.4285   

   242Lys 250Leu Hbond  
0               5.2773  
1               3.9203  
2               3.5174  

[3 rows x 684 columns]

As can be seen above, we have now filtered our dataframe as desired and removed the W50Chi1 column from the dataframe so it is now ready for the stats and ML analysis

## Analysis Time! 

Our dataset is now ready for either statistical analysis or ML or both! 

### Part 1.1. Perform Statistical Analysis with the stat_modelling.py module. 

Now we will perform the actual statistical modelling to compare the differences for each feature against the target variable. 

With this module, we can calculate two different metrics to evaluate how dependant each feature is on the target variables value. They are:

1. The [mutual information](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html) using the implementation available in Scikit-learn. The mutual information can capture any kind of dependancy/relationship between variables and score their dependancy.

2. The [linear correlation](). I assume this does not need an introduction. 

In both cases, the higher the absolute value, the more dependant the feature is on the target variable. The mutual information has scores in the range 0 and 1, whilst the linear correlation scores are in the range -1 to 1. 

In [32]:
# Now time for stat regression analysis
stat_model = stat_modelling.RegressionStatModel(
    dataset=df_ready, # The dataframe we just made. 
    out_dir=stats_out_dir,
    interaction_types_included=["Hbond", "Saltbr", "Hydrophobic", "Other"] 
)

Now we can determine the values for our two metrics, these wont take long to run (maybe 2 mins in total)

In [33]:
stat_model.calc_mutual_info_to_target()

Mutual information scores calculated.
tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/Mutual_Information_Per_Feature_Scores.csv written to disk.
You can also access these results via the class attribute: 'mutual_infos'.


In [34]:
stat_model.calc_linear_correl_to_target()

Linear correlations calculated.
tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/Linear_Correlations_Per_Feature_Scores.csv written to disk.
You can also access these results via the class attribute: 'linear_correlations'.


In [35]:
# As printed above we can access the results from these calculations from the class instance's (we called it stat_model) attributes. 
mi_results = stat_model.mutual_infos
lc_results = stat_model.linear_correlations
# stat_model.__dict__.keys() # uncomment to see all attributes available. 

### Part 1.2. Work up the Statistical Analysis with the post_proccessing.py module. 

In this module we can convert the per feature scores to per residue scores, by summing (and then normalising) every per feature score that each residue is involved in. This can allow us to identify residues which seem to differ the most between each state. 


In [36]:
# First generate an instance of the class. 
post_proc = post_proccessing.StatRegressorPostProcessor(
    stat_model=stat_model,
    out_dir=stats_out_dir
)

In [37]:
# Now we can run the get_per_res_scores() method, changing the stat_method accordingly.
mi_per_res_scores = post_proc.get_per_res_scores(
    stat_method="mutual_information")

lin_correl_per_res_scores = post_proc.get_per_res_scores(
    stat_method="linear_correlation")

tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/Mutual_Information_Scores_Per_Residue.csv written to disk.
tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/Linear_Correlation_Scores_Per_Residue.csv written to disk.


The per feature and per residue scores are not just saved to disk but available as variables so you can analyse them within python using whatever graphing program you like. 

For inspiration feel free to take a look at the article or the tutorial "Tutorial_PTP1B_Classification_ML_Stats.ipynb". 

### Part 1.3. Project the Results onto 3D Protein Structures.  

Naturally, we may want to visualise some of the results we have generated above onto a protein structure. 

We can take advantage of the functions provided in either of the following files: 

1. pymol_projections.py - will output [PyMOL](https://pymol.org/) compatible python scripts
2. chimerax_projections.py will output [ChimeraX](https://www.cgl.ucsf.edu/chimerax/) compatible scripts

Both modules can be used to represent the results at the:
1. Per feature/interaction level. (Cylinders are drawn between each feature, with the cylinder radii marking how strong the relative difference is. 
2. Per residue level. The Carbon alpha of each residue will be depicted as a sphere, with the sphere radii depicting how strong the the relative difference is. 

#### 1.3.1 PyMOL Projections

In [38]:
# Write PyMOL compatable scripts for the per feature results.
# Simply swap between the two statistical methods as shown below. 
pymol_projections.project_pymol_top_features(
    per_feature_scores=stat_model.linear_correlations,
    model_name="linear_correlation",
    numb_features=100, # top features to project, set to any integer or "all" for all features. 
    out_dir=stats_out_dir
)

pymol_projections.project_pymol_top_features(
    per_feature_scores=stat_model.mutual_infos,
    model_name="mutual_information",
    numb_features=100, # top features to project, set to any integer or "all" for all features. 
    out_dir=stats_out_dir
)

The file: tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/linear_correlation_Pymol_Per_Feature_Scores.py was written to disk.
The file: tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/mutual_information_Pymol_Per_Feature_Scores.py was written to disk.


In [39]:
# Write PyMOL compatable scripts for the per residue results.
# Simply swap between the two statistical methods as shown below. 
pymol_projections.project_pymol_per_res_scores(
    per_res_scores=lin_correl_per_res_scores,
    model_name="linear_correlation",
    out_dir=stats_out_dir
)  

pymol_projections.project_pymol_per_res_scores(
    per_res_scores=mi_per_res_scores,
    model_name="mutual_information",
    out_dir=stats_out_dir
)

The file: tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/linear_correlation_Pymol_Per_Res_Scores.py was written to disk.
The file: tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/mutual_information_Pymol_Per_Res_Scores.py was written to disk.


#### 1.3.2 ChimeraX Projections

In [40]:
# Write ChimeraX compatable scripts for the per feature results.
# Simply swap between the two statistical methods as shown below. 
chimerax_projections.project_chimerax_top_features(
    per_feature_scores=stat_model.linear_correlations,
    model_name="linear_correlation",
    pdb_file=pdb_file,
    numb_features=125, # can be any integer values or "all" if you would like all features returned.
    out_dir=stats_out_dir
)

chimerax_projections.project_chimerax_top_features(
    per_feature_scores=stat_model.mutual_infos,
    model_name="mutual_information",
    pdb_file=pdb_file,
    numb_features=125, # can be any integer values or "all" if you would like all features returned.
    out_dir=stats_out_dir
)

The file: tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/linear_correlation_ChimeraX_Per_Feature_Scores.cxc was written to disk.
The file: tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/mutual_information_ChimeraX_Per_Feature_Scores.cxc was written to disk.


In [41]:
# Write ChimeraX compatable scripts for the per residue results.
# Simply swap between the two statistical methods as shown below. 
chimerax_projections.project_chimerax_per_res_scores(
    per_res_scores=lin_correl_per_res_scores,
    model_name="linear_correlation",
    out_dir=stats_out_dir
)

chimerax_projections.project_chimerax_per_res_scores(
    per_res_scores=mi_per_res_scores,
    model_name="mutual_information",
    out_dir=stats_out_dir
)

The file: tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/linear_correlation_ChimeraX_Per_Res_Scores.cxc was written to disk.
The file: tutorial_datasets/KE07_Tutorial/KE07_stat_analysis/mutual_information_ChimeraX_Per_Res_Scores.cxc was written to disk.


Now we are complete with the stats module. Here is an example of the kind of figures you can make:


<center><img src="https://raw.githubusercontent.com/kamerlinlab/KIF/main/tutorials/miscellaneous/ke07_example_outputs.png" style="width: 70%" /></center>

### Part 2.1 Perform Machine Learning (ML) with the model_building.py module. 

Now we will use ML to generate models trained on our target variable. 

With this module, we use the feature importance scores from each ML model to to evaluate how different/similar each feature is against the target variable.

**In the paper we used three ensemble based regression models:**

1. [Categorical Boosting](https://catboost.ai/) - (Refered to as: CatBoost)

2. [Extreme Gradient Boosting](https://xgboost.readthedocs.io/en/stable/) - (Refered to as: XGBoost)

3. [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)  (Refered to as: Random_Forest)

**In this tutorial, we will only use the CatBoost algorithim, to reduce the time required**

In all cases, the higher the score, the more "different" the feature is when in the two different states.

We can use the same dataframe that we used for the statistical analysis in Part 1.1 below.

In [42]:
# Instantiate the ClassificationModel class. 
# Clearly there are many parameters here, using your IDE you can hover over RegressionModel to see what each parameter does. 
ml_model = model_building.RegressionModel(
    dataset=df_ready,
    evaluation_split_ratio=0.15,
    models_to_use=["CatBoost"], # "XGBoost", "Random_Forest"] # You can add the other methods back in you want. 
    scaling_method="min_max",
    out_dir=ml_out_dir, 
    cross_validation_splits=5, 
    cross_validation_repeats=3,
    search_approach="none",
)


Below is a summary of the machine learning you have planned.
You will use 5-fold cross validation and perform 3 repeats.
You will use up to 684 features to build each model, with 85.0% of your data used for training the model, which is 7888 observations. 
15.0% of your data will be used for evaluating the best models produced by the 5-fold cross validation, which is 1393 observations.
You have selected to build 1 machine learning model(s), with the following hyperparameters: 
 
A CatBoost model, with grid search parameters: 
{'iterations': [100]} 

If you're happy with the above, lets get model building!


Now we can go ahead and build the models.

We have one optional parameter in the command below which is to save the models generated. This can be useful if you ever want to come back and do the post-processing later.

If you set this to true all the files required will be saved to a folder called "temporary_files" in your current working directory. 

With the current setup this calculation will not take long (maybe 3 mins to run on a standard laptop). However, you could perform a very exhaustive calculation using grid search CV (possible by changing the "search_approach" parameter in model_building.RegressionModel() ), in which case it might be useful.

In [43]:
ml_model.build_models(save_models=True)

Model saved to disk at: temporary_files/CatBoost_Model.pickle
Model building complete, returning final results with train/test datasets to you.


model          best_params  best_score  best_standard_deviation  \
0  CatBoost  {'iterations': 100}    0.864941                 0.007081   

   Time taken to build model (minutes)  
0                                  0.8

With the model now built, we can see how long it took to build and some metrics describing the regression error for the train and test sets. 

We can now evaluate the quality of the model on the validation dataset (also sometimes refered to as the hold-out set).

In [44]:
reports = ml_model.evaluate_models()
reports

Model  Explained Variance  Mean Absolute Error       MSE     RMSE  \
0  CatBoost              0.8818              18.0938  707.5389  26.5996   

   Mean Squared Log Error  r squared  
0                  0.0413     0.8816

The report produced is a dataframe with 6 regressions metrics to enable you to evaluate the quality of the model. If you had more built more than one model this dataframe would contain additional rows for each additional model built.

The MSE and RMSE stands for the mean squarred error and the root mean squared error respectively.

Personally, I think the Mean Absolute Error (MAE) and RMSE are very useful metrics as they have the same units as your target dataset, meaning it is quite easy to use them to think about how good your model is. 

## Part 2.2. Work up the ML results with the post_proccessing.py module. 

In order to perform the analysis we will need to provide the models generated in Part 2.1 Shown below are the two possible ways to do this. 

In [45]:
# First we will make an instance of the SupervisedPostProcessor class.
ml_post_proc = post_proccessing.SupervisedPostProcessor(
    out_dir=ml_out_dir,
)

# Option 1 - Load models from the instance of the SupervisedModel class. 
ml_post_proc.load_models_from_instance(supervised_model=ml_model)

# Option 2 - Load models from disk. 
# (Use If you've already run the model building, shut down the kernel and now want to post-process).
#ml_post_proc.load_models_from_disk(models_to_use=["XGBoost", "CatBoost", "Random_Forest"]) 

In [46]:
# After preparing the class we can now determine the feature scores for each model made.
ml_post_proc.get_per_feature_scores()

tutorial_datasets/KE07_Tutorial/KE07_ml_analysis/CatBoost_Feature_Scores.csv written to disk.
All per feature scores have now been saved to disk.


In [47]:
# We can also project these per feature scores onto the per-residue level. 
ml_post_proc.get_per_res_scores()

tutorial_datasets/KE07_Tutorial/KE07_ml_analysis/CatBoost_Per_Residue_Scores.csv written to disk.
All per residue scores have now been saved to disk.


In [48]:
print(ml_post_proc.__dict__.keys())

dict_keys(['out_dir', 'feat_names', 'best_models', 'all_per_feature_scores', 'all_per_residue_scores'])


If we take a look at the class attributes we can see the per feature and per residue scores were not just saved to disk, but are also stored inside the class.

This means you could easily analyse them within Python if you want. 

In [49]:
all_per_res_scores = ml_post_proc.all_per_residue_scores
all_feature_scores = ml_post_proc.all_per_feature_scores 

For ideas on how to work up these results please see the manuscript or our other tutorial: "Tutorial_PTP1B_Classification_ML_Stats.ipynb"

### Part 2.3. Project the Results onto Protein Structures 

This section is essentially identical to 1.3, only that now we will output the ml results instead of the stats results

#### 2.3.1 - For projecting the results with PyMOL 
Here you do not need to specify what model you would like to the output results for, all will be outputted simultaneously.

In [50]:
pymol_projections.project_multiple_per_res_scores(
    all_per_res_scores=ml_post_proc.all_per_residue_scores,
    out_dir=ml_out_dir
)

pymol_projections.project_multiple_per_feature_scores(
    all_per_feature_scores=ml_post_proc.all_per_feature_scores,
    numb_features=100, # top features to project, set to any integer or "all" for all features.  
    out_dir=ml_out_dir
)

The file: tutorial_datasets/KE07_Tutorial/KE07_ml_analysis/CatBoost_Pymol_Per_Res_Scores.py was written to disk.
The file: tutorial_datasets/KE07_Tutorial/KE07_ml_analysis/CatBoost_Pymol_Per_Feature_Scores.py was written to disk.


#### 2.3.1 - For projecting the results with ChimeraX
Here you do not need to specify what model you would like to the output results for, all will be outputted simultaneously.

In [51]:
chimerax_projections.project_multiple_per_res_scores(
    all_per_res_scores=ml_post_proc.all_per_residue_scores,
    out_dir=ml_out_dir
)

chimerax_projections.project_multiple_per_feature_scores(
    all_per_feature_scores=ml_post_proc.all_per_feature_scores,
    pdb_file=pdb_file,
    numb_features="all",
    out_dir=ml_out_dir
)

The file: tutorial_datasets/KE07_Tutorial/KE07_ml_analysis/CatBoost_ChimeraX_Per_Res_Scores.cxc was written to disk.
The file: tutorial_datasets/KE07_Tutorial/KE07_ml_analysis/CatBoost_ChimeraX_Per_Feature_Scores.cxc was written to disk.


Done!